In [1]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import glob
import os

# The * is a wildcard that matches any characters
dem_files = glob.glob("P5_PAN_CD*.tif")

# Create a list of rasterio dataset objects to merge
src_files_to_mosaic = []
for fp in dem_files:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

# The merge function combines the rasters
mosaic, out_trans = merge(src_files_to_mosaic)

# Define the metadata for the output file
out_meta = src.meta.copy()
out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                 })

# Write the merged raster to a new file
output_filename = "Kullu_Mandi_DEM_merged.tif"
with rasterio.open(output_filename, "w", **out_meta) as dest:
    dest.write(mosaic)

print(f"Successfully merged {len(dem_files)} files into {output_filename}")

ModuleNotFoundError: No module named 'rasterio'

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np

# Open the merged DEM file
with rasterio.open('Kullu_Mandi_DEM_merged.tif') as dem:
    # Read the elevation data as a numpy array
    # We specify float32 to allow for NaN values
    elevation_data = dem.read(1, masked=False).astype('float32')
    
    # Get the no-data value from the file's metadata
    nodata_value = dem.nodata
    
    # Replace the no-data values with np.nan
    if nodata_value is not None:
        elevation_data[elevation_data == nodata_value] = np.nan
        
    # Get the geographic extent for plotting
    extent = rasterio.plot.plotting_extent(dem)

# Now plot the cleaned data using matplotlib's imshow
fig, ax = plt.subplots(1, 1, figsize=(12, 10))

# Use imshow to display the data. Matplotlib automatically ignores NaN values.
image = ax.imshow(elevation_data, cmap='terrain', extent=extent)

# Add a colorbar to show the elevation scale
cbar = fig.colorbar(image, ax=ax, shrink=0.7)
cbar.set_label('Elevation (meters)')

# Add titles and labels
ax.set_title("Elevation Map of Kullu & Mandi Region", fontsize=16)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
plt.show()

In [ ]:
import rasterio
import pandas as pd

# Part 1: Extract Elevation from Your DEM File 

# Using your successful output as a hardcoded value
# You don't need to run the rasterio part again if you have the value
elevation_value = 1419.93 
print(f"Using pre-calculated elevation: {elevation_value:.2f} meters.")

print("\n--------------------------------------\n")


# Part 2: Load and Prepare Your DataFrames 

print("Loading your weather and events data...")
# Using your specific weather data filename
weather_df = pd.read_csv("HP_Cloudburst_Data_2010-2024.csv")

# Using your specific historical events filename
# FIX 1: Added 'on_bad_lines='skip' to handle the ParserError
events_df = pd.read_csv("himachal_cloudburst_past_data.csv", on_bad_lines='skip')

# Create a standard 'date' column in both dataframes
# FIX 2: Changed to use 'DOY' (Day of Year) instead of 'MO' and 'DY'
weather_df['date'] = pd.to_datetime(weather_df['YEAR'].astype(str) + '-' + weather_df['DOY'].astype(str), format='%Y-%j')
events_df['date'] = pd.to_datetime(events_df['Date'], errors='coerce') # 'errors=coerce' will handle any bad dates in this file too


# Part 3: Combine and Label 

# 1. Add the elevation as a new column (feature)
weather_df['elevation'] = elevation_value

# 2. Create the target column, starting with all zeros
weather_df['is_cloudburst'] = 0

# 3. Use the events list to "label" the correct days with a '1'
# Drop any rows from events_df where the date couldn't be read
events_df = events_df.dropna(subset=['date']) 
event_dates = events_df['date'].tolist()

weather_df.loc[weather_df['date'].isin(event_dates), 'is_cloudburst'] = 1


# Final Step: Save and Verify 

output_filename = "cloud_burst_dataset.csv"
weather_df.to_csv(output_filename, index=False)

print(f"\nSUCCESS! Your final dataset is saved as: {output_filename}")

print("\nVerification:")
print("This shows how many normal days (0) and cloudburst days (1) are in your final dataset.")
# This will show you the count of 0s (normal days) and 1s (cloudburst days)
print(weather_df['is_cloudburst'].value_counts())

In [ ]:
import pandas as pd

try:
    # Load the weather data
    weather_df = pd.read_csv("HP_Cloudburst_Data_2010-2024.csv")
    
    # Load the events data, skipping the bad lines
    events_df = pd.read_csv("himachal_cloudburst_past_data.csv", on_bad_lines='skip')

    # --- Convert Dates (as before) ---
    weather_df['date'] = pd.to_datetime(weather_df['YEAR'].astype(str) + '-' + weather_df['DOY'].astype(str), format='%Y-%j')
    events_df['date'] = pd.to_datetime(events_df['Date'], errors='coerce')

    # --- Diagnostic Step: Check Date Ranges ---
    print("--- Weather Data Date Range ---")
    print(f"Min date: {weather_df['date'].min()}")
    print(f"Max date: {weather_df['date'].max()}")
    print("\n")
    
    # --- Filter for the "Broad Definition" events *before* checking dates ---
    broad_definition_list = [
        'Floods and Heavy Rain', 'Flash floods', 'Cloudburst', 
        'Cloudburst and flash flood', 'Flash flood and landslide', 
        'Flash floods after lake burst', 'Flood', 'Flash flood after cloudburst', 
        'Floods', 'Flash floods due to cloudbursts', 'Flash flood due to lake breach'
    ]
    
    # Drop rows where date couldn't be read
    events_df = events_df.dropna(subset=['date']) 
    # Filter for our events
    cloudburst_events_df = events_df[events_df['Associated_Event'].isin(broad_definition_list)]

    print("--- Event Data (Broad Definition) Date Range ---")
    if not cloudburst_events_df.empty:
        print(f"Min date: {cloudburst_events_df['date'].min()}")
        print(f"Max date: {cloudburst_events_df['date'].max()}")
        print(f"\nFound {len(cloudburst_events_df)} relevant events to check.")
    else:
        print("No events found matching the broad definition.")

except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd
import numpy as np

# This is the single, corrected script.
# It uses the "Broad Definition" AND the date normalization fix.

print("--- Step 1: Loading Raw Data ---")

try:
    # Load the weather data
    weather_df = pd.read_csv("HP_Cloudburst_Data_2010-2024.csv")
    print("Loaded 'HP_Cloudburst_Data_2010-2024.csv'")
    
    # Load the events data, skipping the bad lines
    events_df = pd.read_csv("himachal_cloudburst_past_data.csv", on_bad_lines='skip')
    print("Loaded 'himachal_cloudburst_past_data.csv' (skipped bad lines)")

except FileNotFoundError as e:
    print(f"Error: Could not find a required file: {e}")
    exit()
except Exception as e:
    print(f"An error occurred during loading: {e}")
    exit()

print("\n--- Step 2: Data Preparation & Labeling (Broad Definition) ---")

# --- Date Conversion ---
try:
    weather_df['date'] = pd.to_datetime(weather_df['YEAR'].astype(str) + '-' + weather_df['DOY'].astype(str), format='%Y-%j')
    events_df['date'] = pd.to_datetime(events_df['Date'], errors='coerce')
    print("Converted date columns.")
except KeyError:
    print("Error: Date columns not found.")
    exit()

# --- *** THE CRITICAL FIX *** ---
# Normalize both date columns to remove timestamps and ensure they match.
weather_df['date'] = weather_df['date'].dt.normalize()
events_df['date'] = events_df['date'].dt.normalize()
print("Normalized dates (removed timestamps) for accurate matching.")
# --- *** END OF FIX *** ---

# --- Add Topography Data ---
elevation_value = 1419.93 
weather_df['elevation'] = elevation_value

# --- Labeling the Target Variable ---
weather_df['is_cloudburst'] = 0

# Define the "Broad Definition" list
broad_definition_list = [
    'Floods and Heavy Rain', 'Flash floods', 'Cloudburst', 
    'Cloudburst and flash flood', 'Flash flood and landslide', 
    'Flash floods after lake burst', 'Flood', 'Flash flood after cloudburst', 
    'Floods', 'Flash floods due to cloudbursts', 'Flash flood due to lake breach'
]

# Get a clean list of cloudburst event dates
events_df = events_df.dropna(subset=['date']) 
cloudburst_events_df = events_df[events_df['Associated_Event'].isin(broad_definition_list)]
# Get a list of *unique* dates
event_dates = cloudburst_events_df['date'].unique().tolist()

# Use the events list to "label" the correct days with a '1'
weather_df.loc[weather_df['date'].isin(event_dates), 'is_cloudburst'] = 1

print("\n*** Verification of Labeled Data ***")
print("This output MUST show a count for '1's:")
print(weather_df['is_cloudburst'].value_counts())

# Check how many events we found
num_events = (weather_df['is_cloudburst'] == 1).sum()
if num_events == 0:
    print("\nFATAL ERROR: Still no events found. Stopping.")
    exit()
else:
    print(f"\nSuccessfully labeled {num_events} cloudburst/flash flood event days.")


print("\n--- Step 3: Feature Engineering (Lags & Rolling Averages) ---")

# Set 'date' as the index for time-series operations
df = weather_df.set_index('date')
df = df.sort_index()
print(f"Data sorted by date. Original shape: {df.shape}")

# --- Feature Engineering ---
features_to_lag = ['PRECTOTCORR', 'T2M', 'T2MDEW', 'RH2M', 'WS10M']
lag_periods = [1, 2, 3]
roll_periods = [3, 5]

print("Creating lag features (e.g., 'RH2M_lag_1')...")
for col in features_to_lag:
    for lag in lag_periods:
        new_col_name = f"{col}_lag_{lag}"
        df[new_col_name] = df[col].shift(lag)

print("Creating rolling average features (e.g., 'RH2M_roll_avg_3')...")
for col in features_to_lag:
    for period in roll_periods:
        new_col_name = f"{col}_roll_avg_{period}"
        df[new_col_name] = df[col].shift(1).rolling(window=period).mean()

# --- Data Cleaning ---
print(f"Shape before dropping NaNs: {df.shape}")
df_featured = df.dropna()
print(f"Shape after dropping NaNs: {df_featured.shape}")

# --- Final Verification and Save ---
output_filename = "cloud_burst_final_features.csv"
df_featured.to_csv(output_filename)

print("\n" + "="*50)
print(f"SUCCESS! Your final features dataset is saved as: {output_filename}")
print("This single file is now ready for model training.")
print("\nFinal class balance in the new featured dataset:")
print(df_featured['is_cloudburst'].value_counts())
print("="*50)